In [1]:
import torch
from torch import tensor
import torchvision
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from tqdm import trange
import pandas as pd
from sklearn.metrics import roc_auc_score
import numpy as np

In [2]:
class FairLogReg(nn.Module):
    def __init__(self, D, warm_start = None):
        super(FairLogReg, self).__init__()
        if warm_start is not None:
            self.theta = torch.nn.Parameter(warm_start, requires_grad=True)
        else:
            self.theta = torch.nn.Parameter(torch.zeros(D), requires_grad=True)
        self.old_theta = tensor(float("Inf"))

    def forward(self, x):
        return x.mv(self.theta)


Pick between COMPAS and Adult data sets

In [3]:
# Comment one out

def get_data(filename):
    df = pd.read_csv("data/COMPAS/" + filename + ".csv")
    s = tensor(df['race'] == "Caucasian")
    y = tensor(df['two_year_recid'] == 0).float()
    X = tensor(df.drop(columns=['race','sex','sex-race','two_year_recid']).values).float()
    X = torch.cat((torch.ones(X.shape[0],1), X), dim=1)
    return (X,y,s)
lam_regs = 2. ** np.array([-3, -3, -3, -3, -3])

def get_data(filename):
    df = pd.read_csv("data/Adult/" + filename + ".csv")
    s = tensor(df['sex'] == "Male")
    y = tensor(df['income-per-year'] == ">50K").float()
    X = tensor(df.drop(columns=['sex','race','income-per-year','race-sex','capital-gain', 'capital-loss']).values).float()
    X = torch.cat((torch.ones(X.shape[0],1), X), dim=1)
    return (X,y,s)
lam_regs = 2. ** np.array([-14, -12, -12, -12, -13])

In [4]:
form="linear"
sum_form=1 # 1 for sum, -1 for difference
eoo=False

In [5]:
if form == "logistic":
    def g(outputs):
        return -F.logsigmoid(-outputs).sum()
elif form == "hinge":
    relu = torch.nn.ReLU()
    def g(outputs):
        return relu(outputs+1).sum()
elif form == "linear":
    def g(outputs):
        return outputs.sum()
else:
    raise ValueError("Pick a valid form!")

ploss = nn.BCEWithLogitsLoss()
def floss(outputs, sens_attr, Pa, Pb):
    return sum_form * g(sum_form * outputs[sens_attr])/Pa + g(- outputs[~sens_attr])/Pb

In [6]:
(Xs, ys, ss) = ([None] * 5, [None] * 5, [None] * 5)
(Xts, yts, sts) = ([None] * 5, [None] * 5, [None] * 5)
for i in range(5):
    (Xs[i], ys[i], ss[i]) = get_data("train" + str(i))
    (Xts[i], yts[i], sts[i]) = get_data("test" + str(i))

In [7]:
def make_closure(model, optimizer, lam_fair, lam_reg, X, y, s, Pa, Pb):
    def closure():
        assert not torch.isnan(model.theta).any()
        optimizer.zero_grad()
        outputs = model(X)
        if eoo:
            loss = ploss(outputs,y) + lam_reg * (model.theta**2).mean() + lam_fair/outputs.shape[0] * floss(outputs[y.bool()], s[y.bool()], Pa, Pb)
        else:
            loss = ploss(outputs,y) + lam_reg * (model.theta**2).mean() + lam_fair/outputs.shape[0] * floss(outputs, s, Pa, Pb)
        loss.backward()
        return loss
    return closure

In [8]:
def train_model(X,y,s,lam_fair=0, lam_reg=0, warm_start=None):
    if eoo:
        (Pa, Pb) = ((s & y.bool()).float().mean(), (~s&y.bool()).float().mean())
    else:
        (Pa, Pb) = (s.float().mean(), 1 - s.float().mean())
    model = FairLogReg(X.shape[1], warm_start=warm_start)
    if form == "hingexxx":
        optimizer = optim.Adam(model.parameters(), lr=1)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
    else:
        optimizer = torch.optim.LBFGS(model.parameters(), lr=0.1)
    closure = make_closure(model, optimizer, lam_fair, lam_reg, X, y, s, Pa, Pb)
    for t in trange(500):
        loss = optimizer.step(closure)
        if form == "hingexxx":
            scheduler.step(loss)
        diff = (model.old_theta - model.theta).abs().max()
        if diff < 1e-10:
            break
        model.old_theta = model.theta.clone().detach()
    return (model, t)

In [9]:
def get_summary(model, X,y,s, lam_fair=0, lam_reg=0):
    (Pa, Pb) = (s.float().mean(), 1 - s.float().mean())
    outputs = model(X)
    accuracy = (y == (outputs >= 0)).float().mean()
    if eoo:
        unfairness = (outputs[y.bool() & s] >= 0).float().mean() - (outputs[y.bool() & ~s] >= 0).float().mean()
        relaxation = 1/outputs.shape[0] * floss(outputs[y.bool()], s[y.bool()], Pa, Pb)
    else:
        unfairness = (outputs[s] >= 0).float().mean() - (outputs[~s] >= 0).float().mean()
        relaxation = 1/outputs.shape[0] * floss(outputs, s, Pa, Pb)
    loss = ploss(outputs,y)
    return(accuracy, unfairness, loss, relaxation)

In [10]:
# for weighting baseline, if desired
def get_weighed_loss(X,y,s):
    wobs = y * 10 + s
    wobs[wobs==0.] = (wobs==0.).float().mean()
    wobs[wobs==1.] = (wobs==1.).float().mean()
    wobs[wobs==11.] = (wobs==11.).float().mean()
    wobs[wobs==10.] = (wobs==10.).float().mean()
    wy = (y - (y==0).float().mean()).abs()
    ws = (s.float() - (s==0).float().mean()).abs()
    wexp = ws * wy
    return nn.BCEWithLogitsLoss(weight = (wexp/wobs))

In [11]:
df = pd.DataFrame(columns = ['Split', 'Lam_fair', 'Type', 'Accuracy', 'Unfairness', 'Ploss', 'Relaxation'])
warm_starts = [None] * 5
lfs = np.arange(0, 0.195, 1) #0.02)
for lam_fair in lfs:
    for i in range(5):
        (model,t) = train_model(Xs[i],ys[i],ss[i], lam_fair = lam_fair, lam_reg = lam_regs[i], warm_start=warm_starts[i])
        warm_starts[i] = model.theta.clone().detach()
        (train_accuracy, train_unfairness, train_loss, train_relax) = get_summary(model, Xs[i], ys[i], ss[i], lam_fair = lam_fair, lam_reg = lam_regs[i])
        d = {"Split": i,
             "Type": "Train",
             "Lam_fair": lam_fair.item(),
             'Accuracy': train_accuracy.item(), 
             'Unfairness': train_unfairness.item(),
             'Ploss': train_loss.item(),
             'Relaxation': train_relax.item()}
        df = df.append(d,ignore_index=True)
        (test_accuracy, test_unfairness, test_loss, test_relax) = get_summary(model, Xts[i], yts[i], sts[i], lam_fair = lam_fair, lam_reg = lam_regs[i])
        d = {"Split": i,
             "Type": "Test",
             "Lam_fair": lam_fair.item(),
             'Accuracy': test_accuracy.item(), 
             'Unfairness': test_unfairness.item(),
             'Ploss': test_loss.item(),
             'Relaxation': test_relax.item()}
        df = df.append(d,ignore_index=True)
    print(lam_fair)





 12%|█▏        | 61/500 [00:00<00:06, 64.27it/s]

0.0


In [12]:
df

,Split,Lam_fair,Type,Accuracy,Unfairness,Ploss,Relaxation
0,0,0.0,Train,0.827890,0.081045,0.364729,0.907473
1,0,0.0,Test,0.822383,0.086585,0.377499,0.949487
2,1,0.0,Train,0.827098,0.085512,0.367405,0.900835
3,1,0.0,Test,0.824995,0.074642,0.372423,0.850443
4,2,0.0,Train,0.826702,0.087498,0.368503,0.873033
5,2,0.0,Test,0.827607,0.092553,0.367908,0.951754
6,3,0.0,Train,0.824921,0.086709,0.368838,0.872113
7,3,0.0,Test,0.827004,0.085995,0.368734,0.857507
8,4,0.0,Train,0.828781,0.081810,0.367995,0.847951
9,4,0.0,Test,0.824392,0.100523,0.370887,0.884766
